In [6]:
import pandas as pd
import numpy as np
import ast
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt
df = pd.read_csv("songs_and_chords.csv")
data = [ast.literal_eval(chords_string) for chords_string in df["encoded_chords"]]

Remove too short songs

In [7]:
print(len(data))
SAMPLE_LEN = 8
data = [chords for chords in data if len(chords)>SAMPLE_LEN]
print(len(data))

2137
2132


Sampling dataset 8 chords before and 9'th cord as a value to predict

In [8]:
x = []
y = []
for chords in data:
    for i in range(len(chords)):
        if i < len(chords) - SAMPLE_LEN:
            x.append(chords[i : i + SAMPLE_LEN - 1])
        else:
            x.append(chords[i : len(chords) - 1] + chords[: (i + SAMPLE_LEN) % len(chords)])
        y.append(chords[(i + SAMPLE_LEN) % len(chords)])    
x = np.array(x)
y = np.array(y)
print(x.shape, y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

(103631, 7, 12) (103631, 12)


In [9]:
import music21
for chord in x_samples[0]:
    print(len(chord))
    print(type(list(chord)))
    print(chord)
    # decode
#     music21.chord.Chord([1,2,3])
    break
# names = [music21.chord.Chord(list(chord)).commonName for chord in x_samples[0]]
# print(names)
# print(music21.chord.Chord(y_samples[0]).commonName)

ModuleNotFoundError: No module named 'music21'

Long Short-Term Memory, or LSTM, recurrent neural networks expect three-dimensional input in the Keras Python deep learning library.
LSTMs expect 3D input, and it can be challenging to get your head around this the first time.
LSTMs don’t like sequences of more than 200-400 time steps, so the data will need to be split into samples.

If you have a long sequence of thousands of observations in your time series data, you must split your time series into samples and then reshape it for your LSTM model.

The LSTM needs data with the format of [samples, time steps and features].


- The LSTM input layer must be 3D.
- The meaning of the 3 input dimensions are: samples, time steps, and features.
- The LSTM input layer is defined by the input_shape argument on the first hidden layer.
- The input_shape argument takes a tuple of two values that define the number of time steps and features.
- The number of samples is assumed to be 1 or more.
- The reshape() function on NumPy arrays can be used to reshape your 1D or 2D data to be 3D.
- The reshape() function takes a tuple as an argument that defines the new shape.


In [ ]:

# LSTM for sequence classification in the IMDB dataset
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence


# create the model
model = Sequential()
model.add(LSTM(32, input_shape=(7, 12)))
model.add(Dense(12, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=8)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(loaded_model.summary())
scores = loaded_model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 32)                5760      
_________________________________________________________________
dense_6 (Dense)              (None, 12)                396       
Total params: 6,156
Trainable params: 6,156
Non-trainable params: 0
_________________________________________________________________
None
